
# Algoritmo de triangulação

A triangulação do polígono é o primeiro ingrediente da nossa solução para o problema da galeria de arte. O objetivo desse notebook é explicar como fazer isso através do algoritmo de *ear decomposition* (decomposição por orelhas).  
Primeiro, lemos o polígono de um arquivo de entrada

In [ ]:
import matplotlib.pyplot as plt

# Selecione o polígono entre in0, in1, in2 e in3
polygon = "in2"

# Leitura da entrada

with open(polygon) as f:
    L = list(f.readline().split())

N = int(L[0])
L = L[1:]

points = []
for i in range(0, N):
    s, t = L[2*i], L[2*i+1]
    a, b = map(int, s.split('/'))
    c, d = map(int, t.split('/'))
    points.append((a/b, c/d))

xs = [x for (x, y) in points]
ys = [y for (x, y) in points]
plt.figure(figsize=(8, 8))
plt.axis('equal')
plt.fill(xs, ys)
plt.show()

Precisamos de algumas primitivas geométricas

In [ ]:
# Primitiva de sentido horário/anti-horário

eps = 1e-6

def ccw(i, j, k):
    (ix, iy) = points[i]
    (jx, jy) = points[j]
    (kx, ky) = points[k]
    
    (vx, vy) = (jx - ix, jy - iy)
    (wx, wy) = (kx - jx, ky - jy)
    
    return vx*wy - vy*wx > eps


# Primitiva de estar dentro de um triângulo (l dentro de i, j, k)

def inside(i, j, k, l):
    return ccw(i, j, l) and ccw(j, k, l) and ccw(k, i, l)

Segue o algoritmo que decide se um ponto é uma orelha

In [ ]:
# Sou ponta de orelha? Complexidade O(N)

def eartest(j, verbose=False):
    i, k = prev[j], prox[j]
    
    if not ccw(i, j, k):
        if verbose:
            print(j, "is not an ear because", i, j, k, "are not counterclockwise")
        return False
    
    for l in range(0, N):
        if inside(i, j, k, l):
            if verbose:
                print(j, "is not an ear because", l, "is in triangle", i, j, k)
            return False
    
    return True

Segue o algoritmo de triangulação

In [ ]:
# Adjacência no polígono

prev = []
prox = []

for i in range(0, N):
    prev.append((i-1)%N)
    prox.append((i+1)%N)
  
    
# Inicialmente, quem é orelha? Complexidade: O(N^2)

ear = []

for j in range(0, N):
    ear.append(eartest(j))


# Triangulação. Complexidade: O(N^2)

triangulation = []
remaining = N

while remaining >= 3:

    # Encontrar uma orelha
    i = 0
    for j in range(0, N):
        if ear[j]:
            i = j
            break
    
    # Adicionar o triangulo encontrado
    triangulation.append((prev[i], i, prox[i]))
    ear[i] = False
    
    # Remover i do polígono
    prox[prev[i]] = prox[i]
    prev[prox[i]] = prev[i]
    remaining -= 1
    
    # Atualizar "status de orelha" dos vizinhos
    ear[prev[i]] = eartest(prev[i])
    ear[prox[i]] = eartest(prox[i])


for t in triangulation:
    l = [points[i] for i in t]
    xs = [x for (x, y) in l]
    ys = [y for (x, y) in l]
    plt.fill(xs, ys, facecolor='red', edgecolor='blue', linewidth=0.3)
plt.show()